# Restricted Two Body Problem: Elliptical Orbits Around a Central Mass

This is the general case of a Keperian orbit.<br>
A light body (e.g. a planet) orbits a heavy central body (e.g. the sun).  The orbit is an ellipse with the primary at one focus.

In [1]:
# Library imports
import tensorflow as tf
import rebound
import numpy as np

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory, TimeHistory
from tf_utils import plot_loss_hist, EpochLoss, TimeHistory
from tf_utils import Identity

# from orbital_element import make_data_orb_elt
from r2b_data import make_traj_r2b, make_train_r2b

In [3]:
# Generate one example trajectory
a = 1.0
e = 0.0
inc = 0.0
Omega = 0.0
omega = 0.0
f = 0.0
n_years = 2

inputs_traj, outputs_traj = make_traj_r2b(a=a, e=e, inc=inc, Omega=Omega, omega=omega, f=f, n_years=n_years)

In [4]:
inputs_traj.keys()

dict_keys(['t', 'q0', 'v0', 'mu'])

In [ ]:
def make_train_r2b(n_traj: int, n_years: int, a_min: float = 0.50, a_max: float = 32.0, 
                   e_max = 0.20, inc_max = 0.0, seed = 42):
    """
    Make a set of training data for the restricted two body problem
    INPUTS:
    n_traj: the number of trajectories to sample
    n_years: the number of years for each trajectory, e.g. 2
    a_min: minimal semi-major axis in AU, e.g. 0.50
    a_max: maximal semi-major axis in AU, e.g. 32.0
    e_max: maximum eccentricity, e.g. 0.20
    """
    # The sample frequency is 365 per year (dt approximately = 1 day)
    sample_freq = 365
    # Number of samples including both start and end in each trajectory
    traj_size = sample_freq*n_years + 1
    # Number of spatial dimensions
    space_dims = 3

    # Shape of arrays for various inputs and outputs
    scalar_shape = (n_traj, 1)
    init_shape = (n_traj, space_dims)
    time_shape = (n_traj, traj_size)
    traj_shape = (n_traj, traj_size, space_dims)
    
    # Set random seed for reproducible results
    np.random.seed(seed=seed)

    # Initialize orbital element by sampling according to the inputs
    orb_a = np.random.uniform(low=a_min, high=a_max, size=n_traj).astype(np.float32)
    orb_e = np.random.uniform(low=0.0, high=e_max, size=n_traj).astype(np.float32)
    orb_inc = np.random.uniform(low=0.0, high=inc_max, size=n_traj).astype(np.float32)
    orb_Omega = np.random.uniform(low=-np.pi, high=np.pi, size=n_traj).astype(np.float32)
    orb_omega = np.random.uniform(low=-np.pi, high=np.pi, size=n_traj).astype(np.float32)
    orb_f = np.random.uniform(low=-np.pi, high=np.pi, size=n_traj).astype(np.float32)

    # Initialize arrays for the data
    t = np.zeros(time_shape, dtype=np.float32)
    q0 = np.zeros(init_shape, dtype=np.float32)
    v0 = np.zeros(init_shape, dtype=np.float32)
    mu = np.zeros(scalar_shape, dtype=np.float32)
    q = np.zeros(traj_shape, dtype=np.float32)
    v = np.zeros(traj_shape, dtype=np.float32)
    a = np.zeros(traj_shape, dtype=np.float32)
    T = np.zeros(time_shape, dtype=np.float32)
    U = np.zeros(time_shape, dtype=np.float32)
    H = np.zeros(time_shape, dtype=np.float32)
    L = np.zeros(traj_shape, dtype=np.float32)
    
    # Sample the trajectories
    for i in range(n_traj):
        # Generate one trajectory
        inputs, outputs = make_traj_r2b(a=orb_a[i], e=orb_e[i], inc=orb_inc[i], 
                                        Omega=orb_Omega[i], omega=orb_omega[i], 
                                        f=orb_f[i], n_years=n_years)
        
        # Copy results into main arrays
        t[i, :] = inputs['t']
        q0[i, :] = inputs['q0']
        v0[i, :] = inputs['v0']
        mu[i, :] = inputs['mu']
        q[i, :, :] = outputs['q']
        v[i, :, :] = outputs['v']
        a[i, :, :] = outputs['a']
        T[i, :] = outputs['T']
        U[i, :] = outputs['U']
        H[i, :] = outputs['H']
        L[i, :] = outputs['L']

    # Assemble the input dict
    inputs = {
        't': t,
        'q0': q0,
        'v0': v0,
        'mu': mu}

    # Assemble the output dict
    outputs = {
        'q': q,
        'v': v,
        'a': a,
        'q0_rec': q0,
        'v0_rec': v0,
        'T': T,
        'U': U,
        'H': H,
        'L': L}

    # Return the dicts
    return (inputs, outputs)

In [ ]:
# Test make_train_r2b(n_traj: int, n_years: int, a_min: float = 0.25, a_max: float = 32.0, e_max = 0.20, seed = 42):
n_traj = 100
n_years = 2
a_min = 0.50
a_max = 32.0
e_max = 0.20
inc_max = np.pi/4.0
seed = 42

inputs, outputs= make_train_r2b(n_traj=n_traj, n_years=n_years, a_min=a_min, a_max=a_max, 
                                e_max=e_max, inc_max=inc_max, seed=seed)